In [3]:
!pip install mediapipe opencv-python pandas scikit-learn

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [25]:
#import statements
import mediapipe as mp
import cv2
from sklearn.metrics import accuracy_score
import pickle
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
#Get the drawings utilitys and model
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic 

In [5]:
#Webcam Feed
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        
        
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))

        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
          
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        cv2.imshow("Raw Webcam Feed", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
    cap.release()
    cv2.destroyAllWindows()


In [3]:
import csv
import os
import numpy as np

In [6]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)

In [16]:
landmarks = ['class']
for (val) in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [47]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [41]:
class_name = "Normal"

In [42]:
#Webcam Feed
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))

        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
          
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row
            row.insert(0,class_name)
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass
        
        cv2.imshow("Raw Webcam Feed", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
            
    cap.release()
    cv2.destroyAllWindows()

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
df = pd.read_csv('coords.csv')

In [44]:
x = df.drop('class',axis =1) #features
y = df['class']  # Target Value

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3,random_state=2342)

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [46]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [47]:
fit_models={}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train,y_train)
    fit_models[algo] = model

c:\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [35]:
from sklearn.metrics import accuracy_score
import pickle

In [49]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test,yhat))

lr 0.9984825493171472
rc 0.9939301972685888
rf 0.9969650986342944
gb 0.992412746585736


In [50]:
with open('body_language_rf.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'],f)

In [51]:
with open('body_language_rf.pkl', 'rb') as f:
    model = pickle.load(f)

In [52]:
#Webcam Feed
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))

        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
          
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row
            x_coord = pd.DataFrame([row])
            body_language_class = model.predict(x_coord)[0]
            body_language_prob = model.predict_proba(x_coord)[0]
            #print(body_language_class,body_language_prob)
            
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                                               ), [640,480]).astype(int))
            
            cv2.rectangle(image, (coords[0], coords[1]+5), (coords[0]+len(body_language_class)*20, coords[1]-30), (245,117,16),-1)
    
            cv2.putText(image, body_language_class, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)
         
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)
            cv2.putText(image, "CLASS", (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (95,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
           

            cv2.putText(image, "PROB", (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2)), (10,40), cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255), 2, cv2.LINE_AA)
            
        except:
            pass
        
        cv2.imshow("Raw Webcam Feed", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
            
    cap.release()
    cv2.destroyAllWindows()